In [1]:
import pandas as pd
import numpy as np
import sklearn

import warnings
#warnings.filterwarnings("ignore")

from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score

from copy import deepcopy

import xgboost as xgb

N_FOLDS = 3

In [2]:
df_train  = pd.read_csv('./ngwl-predict-customer-churn/train/train.csv')
#df_train
df_train['order_completed_at'] = pd.to_datetime(df_train['order_completed_at'])

df_test = pd.read_csv("./ngwl-predict-customer-churn/sample_submission.csv", sep=';')
df_test

,Id,Predicted
0,19843,NaN
1,471287,NaN
2,342522,NaN
3,457799,NaN
4,233778,NaN
...,...,...
214604,16841,NaN
214605,69348,NaN
214606,594974,NaN
214607,320024,NaN


In [3]:
train_data_def = pd.read_csv('./ngwl-predict-customer-churn/train/train.csv', sep=',')
#print(train_data_def)

train_start = '2020-06'
test_start = '2020-07'

train_data = train_data_def[train_data_def['order_completed_at']==train_start]
print(train_data['phone_id'].nunique())
print(train_data['target'].value_counts())

test_data = train_data_def[train_data_def['order_completed_at']==test_start]
print(test_data['phone_id'].nunique())
print(test_data['target'].value_counts())

train_data.loc[:, 'fold_num'] = [np.random.randint(1,N_FOLDS+1) for i in range(len(train_data))]
train_data['fold_num'].value_counts()
train_data['month']=6

test_data.loc[:, 'fold_num'] = [np.random.randint(1,N_FOLDS+1) for i in range(len(test_data))]
test_data['fold_num'].value_counts()
test_data['month']=7


train_folds = []
test_folds = []

for i in range(1, N_FOLDS+1):
    local_data_train = train_data.loc[train_data['fold_num']==i]
    local_data_test = test_data.loc[test_data['fold_num']==i]

    del local_data_train['fold_num']
    del local_data_test['fold_num']
    
    del local_data_train['order_completed_at']
    del local_data_test['order_completed_at']
    
    train_folds.append(local_data_train.reset_index(drop=True))
    test_folds.append(local_data_test.reset_index(drop=True))

254285
0.0    127797
1.0    126488
Name: target, dtype: int64
284449
0.0    151807
1.0    132642
Name: target, dtype: int64


/home/dana_zl/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/dana_zl/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/home/dana_zl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [4]:
train_data_def = pd.read_csv('./ngwl-predict-customer-churn/train/train.csv', sep=',')
#print(train_data_def)

train_start_prev = '2020-05'
test_start_prev = '2020-06'

train_data_prev = train_data_def[train_data_def['order_completed_at']==train_start_prev]
print(train_data_prev['phone_id'].nunique())
print(train_data_prev['target'].value_counts())

test_data_prev = train_data_def[train_data_def['order_completed_at']==test_start_prev]
print(test_data_prev['phone_id'].nunique())
print(test_data_prev['target'].value_counts())

train_data_prev.loc[:, 'fold_num'] = [np.random.randint(1,N_FOLDS+1) for i in range(len(train_data_prev))]
train_data_prev['fold_num'].value_counts()
train_data_prev['month']=6

test_data_prev.loc[:, 'fold_num'] = [np.random.randint(1,N_FOLDS+1) for i in range(len(test_data_prev))]
test_data_prev['fold_num'].value_counts()
test_data_prev['month']=7


train_folds_prev = []
test_folds_prev = []

for i in range(1, N_FOLDS+1):
    local_data_train = train_data_prev.loc[train_data_prev['fold_num']==i]
    local_data_test = test_data_prev.loc[test_data_prev['fold_num']==i]

    del local_data_train['fold_num']
    del local_data_test['fold_num']
    
    del local_data_train['order_completed_at']
    del local_data_test['order_completed_at']
    
    train_folds_prev.append(local_data_train.reset_index(drop=True))
    test_folds_prev.append(local_data_test.reset_index(drop=True))

218569
1.0    119923
0.0     98646
Name: target, dtype: int64
254285
0.0    127797
1.0    126488
Name: target, dtype: int64


/home/dana_zl/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/dana_zl/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/home/dana_zl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [5]:
# shipments
shipments = pd.DataFrame({})
for i in ['shipments2020-01-01', 'shipments2020-03-01', 'shipments2020-04-30', 'shipments2020-06-29']:
    tmp = pd.read_csv(f'ngwl-predict-customer-churn/shipments/{i}.csv')
    shipments = pd.concat([tmp, shipments])
    
adress = pd.read_csv('ngwl-predict-customer-churn/misc/addresses.csv')
# add users phone
shipments = shipments.merge(adress, how='left', left_on='ship_address_id', right_on='id')

# month
shipments['dt_'] = pd.to_datetime(shipments['order_completed_at'], format='%Y-%m-%d %H:%M:%S')
shipments = shipments[shipments['dt_']>='2020-01-01']
shipments['month'] = pd.to_datetime(shipments['order_completed_at'], format='%Y-%m-%d %H:%M:%S').dt.month
shipments = shipments.drop_duplicates()
#shipments_first = shipments.groupby(by='phone_id', as_index=False)['dt_'].min()
#shipments_first['first_month'] = shipments_first['dt_'].dt.month

In [6]:
shipments['rate'] = shipments['rate'].replace({0:3})
f_shipments = shipments.groupby(by=['phone_id', 'month'], as_index=False).agg({
    'rate': ['mean'],

})
f_shipments.columns = [x[0]+'_'+x[1] + "_new" if 
                       not ((x[0] =='phone_id') or (x[0] =='month'))
                       else x[0]
                       for x in f_shipments.columns]
f_shipments

,phone_id,month,rate_mean_new
0,2,1,3.0
1,2,2,3.0
2,2,3,3.0
3,2,5,3.0
4,3,3,5.0
...,...,...,...
1155779,719367,8,3.0
1155780,719369,8,5.0
1155781,719370,8,5.0
1155782,719373,8,3.0


In [7]:
client_features = pd.read_pickle("./fea_all_actual_01_38.pkl")
client_features

,order_id_nunique,total_cost_sum,total_cost_min,total_cost_max,total_cost_mean,total_cost_std,rate_mean,promo_total_sum,promo_total_mean,total_weight_sum,...,Москва,Санкт-Петербург,discount_msg_sum,bonus_msg_sum,spam_msg_sum,email_sum,push_sum,sms_sum,action_id_nunique,gap_in_month_before
0,1,98.0,98.0,98.0,98.0,0.000000,0.0,0.000000,0.000000,22024,...,1,0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1
1,2,366.0,158.0,208.0,183.0,35.355339,0.0,0.000000,0.000000,67946,...,1,0,0.0,0.0,3.0,3.0,0.0,0.0,3.0,0
2,1,233.0,233.0,233.0,233.0,0.000000,0.0,0.000000,0.000000,34195,...,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,1,248.0,248.0,248.0,248.0,0.000000,0.0,0.000000,0.000000,40777,...,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,2,0.0,0.0,0.0,0.0,0.000000,5.0,0.000000,0.000000,31265,...,1,0,0.0,0.0,3.0,3.0,0.0,0.0,3.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1155779,1,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,5320,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1155780,1,0.0,0.0,0.0,0.0,0.000000,5.0,0.000000,0.000000,12270,...,0,0,1.0,3.0,12.0,6.0,8.0,2.0,16.0,1
1155781,1,0.0,0.0,0.0,0.0,0.000000,5.0,-163.770004,-163.770004,14039,...,0,1,0.0,4.0,14.0,9.0,9.0,2.0,18.0,1
1155782,1,0.0,0.0,0.0,0.0,0.000000,0.0,-193.270004,-193.270004,18470,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [8]:
client_features = client_features.merge(f_shipments, on=['month', 'phone_id'])
client_features

,order_id_nunique,total_cost_sum,total_cost_min,total_cost_max,total_cost_mean,total_cost_std,rate_mean,promo_total_sum,promo_total_mean,total_weight_sum,...,Санкт-Петербург,discount_msg_sum,bonus_msg_sum,spam_msg_sum,email_sum,push_sum,sms_sum,action_id_nunique,gap_in_month_before,rate_mean_new
0,1,98.0,98.0,98.0,98.0,0.000000,0.0,0.000000,0.000000,22024,...,0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1,3.0
1,2,366.0,158.0,208.0,183.0,35.355339,0.0,0.000000,0.000000,67946,...,0,0.0,0.0,3.0,3.0,0.0,0.0,3.0,0,3.0
2,1,233.0,233.0,233.0,233.0,0.000000,0.0,0.000000,0.000000,34195,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,3.0
3,1,248.0,248.0,248.0,248.0,0.000000,0.0,0.000000,0.000000,40777,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3.0
4,2,0.0,0.0,0.0,0.0,0.000000,5.0,0.000000,0.000000,31265,...,0,0.0,0.0,3.0,3.0,0.0,0.0,3.0,1,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1155779,1,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,5320,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3.0
1155780,1,0.0,0.0,0.0,0.0,0.000000,5.0,0.000000,0.000000,12270,...,0,1.0,3.0,12.0,6.0,8.0,2.0,16.0,1,5.0
1155781,1,0.0,0.0,0.0,0.0,0.000000,5.0,-163.770004,-163.770004,14039,...,1,0.0,4.0,14.0,9.0,9.0,2.0,18.0,1,5.0
1155782,1,0.0,0.0,0.0,0.0,0.000000,0.0,-193.270004,-193.270004,18470,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3.0


In [11]:
Counter(client_features['rate_mean_new'])

Counter({3.0: 407106,
         5.0: 447912,
         1.0: 6165,
         4.333333333333333: 34259,
         3.5: 15542,
         3.6666666666666665: 22730,
         2.0: 4807,
         4.0: 129177,
         4.5: 26506,
         4.571428571428571: 414,
         4.285714285714286: 353,
         4.666666666666667: 8648,
         4.2: 4890,
         3.875: 110,
         4.142857142857143: 888,
         3.611111111111111: 3,
         4.6: 7380,
         3.3333333333333335: 3441,
         4.75: 3588,
         4.777777777777778: 574,
         4.555555555555555: 440,
         4.8: 1795,
         4.857142857142857: 467,
         4.266666666666667: 7,
         4.733333333333333: 24,
         3.8: 2923,
         3.111111111111111: 16,
         4.714285714285714: 1859,
         4.125: 147,
         4.222222222222222: 100,
         4.6923076923076925: 69,
         3.888888888888889: 185,
         4.9: 68,
         3.4: 1832,
         3.8181818181818183: 27,
         4.111111111111111: 251,
        

In [9]:
from collections import Counter

In [5]:
import gc
del client_features
gc.collect()

20

In [9]:
client_features_6 = client_features[client_features['month'] == 6]
client_features_7 = client_features[client_features['month'] == 7]
client_features_56 = pd.merge(client_features[client_features['month'] == 5],
                              client_features[client_features['month'] == 6],
         how='outer', on='phone_id')

client_features_34 = pd.merge(client_features[client_features['month'] == 3],
                              client_features[client_features['month'] == 4],
                              how='outer', on='phone_id'
) 

client_features_45 = pd.merge(client_features[client_features['month'] == 4],
                              client_features[client_features['month'] == 5],
                              how='outer', on='phone_id'
)   
client_features_67 = pd.merge(
    client_features[client_features['month'] == 6],
    client_features[client_features['month'] == 7],
    how='outer', on='phone_id'
)

client_features_78 = pd.merge(
    client_features[client_features['month'] == 7],
    client_features[client_features['month'] == 8],
    how='outer', on='phone_id'
)  

client_features_345 = pd.merge(
    client_features_34,
    client_features[client_features['month'] == 5],
    how='outer', on='phone_id'
)

client_features_678 = pd.merge(
    client_features_67,
    client_features[client_features['month'] == 8],
    how='outer', on='phone_id'
)
                              
client_features_567 = pd.merge(client_features_56, client_features[client_features['month'] == 7],
         how='outer', on='phone_id')
client_features_456 = pd.merge(client_features_45, client_features[client_features['month'] == 6],
         how='outer', on='phone_id')

client_features_456['delta_order_cnt_12'] = client_features_456['order_id_nunique_y'] - client_features_456['order_id_nunique_x']
client_features_456['delta_order_cnt_23'] = client_features_456['order_id_nunique'] - client_features_456['order_id_nunique_y']
client_features_456['delta_order_state_complete_12'] = client_features_456['order_state_complete_y'] \
- client_features_456['order_state_complete_x']
client_features_456['delta_order_state_complete_23'] = client_features_456['order_state_complete'] \
- client_features_456['order_state_complete_y']
client_features_456['delta_price_sum_12'] = client_features_456['price_sum_y'] \
- client_features_456['price_sum_x']
client_features_456['delta_price_sum_23'] = client_features_456['price_sum'] \
- client_features_456['price_sum_y']

client_features_567['delta_order_cnt_12'] = client_features_567['order_id_nunique_y'] - client_features_567['order_id_nunique_x']
client_features_567['delta_order_cnt_23'] = client_features_567['order_id_nunique'] - client_features_567['order_id_nunique_y']
client_features_567['delta_order_state_complete_12'] = client_features_567['order_state_complete_y'] \
- client_features_567['order_state_complete_x']
client_features_567['delta_order_state_complete_23'] = client_features_567['order_state_complete'] \
- client_features_567['order_state_complete_y']
client_features_567['delta_price_sum_12'] = client_features_567['price_sum_y'] \
- client_features_567['price_sum_x']
client_features_567['delta_price_sum_23'] = client_features_567['price_sum'] \
- client_features_567['price_sum_y']

client_features_678['delta_order_cnt_12'] = client_features_678['order_id_nunique_y'] - client_features_678['order_id_nunique_x']
client_features_678['delta_order_cnt_23'] = client_features_678['order_id_nunique'] - client_features_678['order_id_nunique_y']
client_features_678['delta_order_state_complete_12'] = client_features_678['order_state_complete_y'] \
- client_features_678['order_state_complete_x']
client_features_678['delta_order_state_complete_23'] = client_features_678['order_state_complete'] \
- client_features_678['order_state_complete_y']
client_features_678['delta_price_sum_12'] = client_features_678['price_sum_y'] \
- client_features_678['price_sum_x']
client_features_678['delta_price_sum_23'] = client_features_678['price_sum'] \
- client_features_678['price_sum_y']

client_features_567

,order_id_nunique_x,total_cost_sum_x,total_cost_min_x,total_cost_max_x,total_cost_mean_x,total_cost_std_x,rate_mean_x,promo_total_sum_x,promo_total_mean_x,total_weight_sum_x,...,sms_sum,action_id_nunique,gap_in_month_before,rate_mean_new,delta_order_cnt_12,delta_order_cnt_23,delta_order_state_complete_12,delta_order_state_complete_23,delta_price_sum_12,delta_price_sum_23
0,1.0,248.0,248.0,248.0,248.0,0.000000,0.00,0.0,0.0,40777.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,168.0,168.0,168.0,168.0,0.000000,1.00,-250.0,-250.0,25106.0,...,0.0,25.0,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,331.0,163.0,168.0,165.5,3.535534,2.00,0.0,0.0,56605.0,...,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,-2924.440041,NaN
3,4.0,316.0,0.0,158.0,79.0,91.221343,1.25,-600.0,-150.0,57316.0,...,0.0,1.0,0.0,3.0,-2.0,1.0,-2.0,1.0,-9982.900042,6533.700037
4,1.0,158.0,158.0,158.0,158.0,0.000000,5.00,0.0,0.0,3990.0,...,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,1388.239992,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,7.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
404033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,14.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
404034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,22.0,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN
404035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,3.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
client_features.columns

Index(['order_id_nunique', 'total_cost_sum', 'total_cost_min',
       'total_cost_max', 'total_cost_mean', 'total_cost_std', 'rate_mean',
       'promo_total_sum', 'promo_total_mean', 'total_weight_sum',
       'total_weight_mean', 'month', 'phone_id', 'month_after_first', 'age',
       'genger_nan', 'gender_male', 'gender_female', 'order_state_complete',
       'order_state_canceled', 'order_state_resumed', 'order_state_cart',
       'shipment_state_shipped', 'shipment_state_canceled',
       'shipment_state_collecting', 'shipment_state_ready',
       'shipment_state_shipping', 'shipment_state_ready_to_ship',
       'shipment_state_pending', 'discount_sum', 'quantity_sum',
       'item_id_nunique_sum', 'price_sum', 'price_std_mean', 'cancelled_pr',
       'replaced_pr', 'day_after_last', 'android', 'ios', 'linux', 'mac',
       'other', 'windows', 'Other_city', 'Москва', 'Санкт-Петербург',
       'discount_msg_sum', 'bonus_msg_sum', 'spam_msg_sum', 'email_sum',
       'push_sum', 'sms

In [39]:
from collections import Counter

In [15]:
train_features

,phone_id,target,month_x,order_id_nunique_x,total_cost_sum_x,total_cost_min_x,total_cost_max_x,total_cost_mean_x,total_cost_std_x,rate_mean_x,...,Москва,Санкт-Петербург,discount_msg_sum,bonus_msg_sum,spam_msg_sum,email_sum,push_sum,sms_sum,action_id_nunique,gap_in_month_before
0,19843,0.0,6,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,173074,0.0,6,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,68581,0.0,6,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,11.0,3.0,8.0,0.0,11.0,0.0
3,50685,1.0,6,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,8.0,0.0,8.0,0.0,8.0,0.0
4,82711,1.0,6,1.0,98.0,98.0,98.0,98.0,0.00000,5.0,...,0.0,0.0,0.0,0.0,13.0,0.0,13.0,0.0,11.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75230,66542,0.0,6,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75231,116401,0.0,6,1.0,168.0,168.0,168.0,168.0,0.00000,5.0,...,1.0,0.0,0.0,0.0,13.0,2.0,11.0,0.0,13.0,0.0
75232,49252,1.0,6,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75233,106273,1.0,6,3.0,1155.0,158.0,723.0,385.0,298.40744,0.0,...,1.0,0.0,0.0,0.0,14.0,0.0,14.0,0.0,13.0,0.0


In [16]:
train_features_prev

,phone_id,target,month_x,order_id_nunique_x,total_cost_sum_x,total_cost_min_x,total_cost_max_x,total_cost_mean_x,total_cost_std_x,rate_mean_x,...,Москва,Санкт-Петербург,discount_msg_sum,bonus_msg_sum,spam_msg_sum,email_sum,push_sum,sms_sum,action_id_nunique,gap_in_month_before
0,615032,0.0,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,16.0,0.0,16.0,0.0,16.0,0.0
1,40738,1.0,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,174054,1.0,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,17.0,3.0,14.0,0.0,17.0,0.0
3,31838,0.0,6,1.0,158.0,158.0,158.0,158.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,86829,1.0,6,1.0,102.0,102.0,102.0,102.0,0.0,0.0,...,0.0,0.0,0.0,0.0,13.0,1.0,12.0,0.0,13.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65387,106273,1.0,6,1.0,188.0,188.0,188.0,188.0,0.0,0.0,...,1.0,0.0,0.0,0.0,19.0,0.0,19.0,0.0,17.0,0.0
65388,238680,1.0,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,15.0,2.0,13.0,0.0,13.0,1.0
65389,69348,1.0,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,4.0,2.0,2.0,0.0,4.0,1.0
65390,137908,0.0,6,1.0,149.0,149.0,149.0,149.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
pd.concat([train_features, train_features_prev])

,phone_id,target,month_x,order_id_nunique_x,total_cost_sum_x,total_cost_min_x,total_cost_max_x,total_cost_mean_x,total_cost_std_x,rate_mean_x,...,Москва,Санкт-Петербург,discount_msg_sum,bonus_msg_sum,spam_msg_sum,email_sum,push_sum,sms_sum,action_id_nunique,gap_in_month_before
0,19843,0.0,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,173074,0.0,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,68581,0.0,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.0,3.0,8.0,0.0,11.0,0.0
3,50685,1.0,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,8.0,0.0,8.0,0.0,8.0,0.0
4,82711,1.0,6,1.0,98.0,98.0,98.0,98.0,0.0,5.0,...,0.0,0.0,0.0,0.0,13.0,0.0,13.0,0.0,11.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65387,106273,1.0,6,1.0,188.0,188.0,188.0,188.0,0.0,0.0,...,1.0,0.0,0.0,0.0,19.0,0.0,19.0,0.0,17.0,0.0
65388,238680,1.0,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,15.0,2.0,13.0,0.0,13.0,1.0
65389,69348,1.0,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,4.0,2.0,2.0,0.0,4.0,1.0
65390,137908,0.0,6,1.0,149.0,149.0,149.0,149.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
#clf = SGDClassifier()
clf = xgb.XGBClassifier(n_estimators=400, max_depth=6, learning_rate=0.05)

for idx in range(2):
    train_fold = train_folds[idx]
    test_fold = test_folds[idx]
    
    train_fold_prev = train_folds_prev[idx]
    #test_fold = test_folds[idx]
        
    train_features = pd.merge(train_fold, client_features_456, on='phone_id').fillna(0)
    print(len(train_features))
    train_features_prev = pd.merge(train_fold_prev, client_features_345, on='phone_id').fillna(0)
    train_features = pd.concat([train_features, train_features_prev])
    print(len(train_fold), len(train_features), len(train_features_prev),train_features.shape)
       
    #print(train_features)
    train_target = deepcopy(train_features['target'])
    del train_features['target']
    del train_features['phone_id']
    del train_features['month_x']
    del train_features['month_y']
    train_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y'], axis=1, inplace=True)

    
    test_features = pd.merge(test_fold, client_features_567, on='phone_id').fillna(0)
    #test_features = pd.merge(test_fold, client_features_7, on='phone_id').fillna(0)
    print(len(test_fold), len(test_features), test_features.shape)
    
    test_target = deepcopy(test_features['target'])
    del test_features['target']
    del test_features['phone_id']
    del test_features['month_x']
    del test_features['month_y']
    test_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y'], axis=1, inplace=True)
    
    #print(train_features.columns, train_target)
    #print(test_features.columns, test_target)

    clf.fit(train_features, train_target)
    print(type(clf))
    new_preds = clf.predict(test_features)
    score = f1_score(test_target, new_preds)
    print(score)

75235
84780 140627 65392 (140627, 162)
94986 83322 (83322, 162)
<class 'xgboost.sklearn.XGBClassifier'>
0.70923233868911
75564
84809 140510 64946 (140510, 162)
94893 83336 (83336, 162)


KeyboardInterrupt: 

In [8]:
for x in zip(clf.feature_importances_, test_features.columns):
    print(x)

(0.0025180755, 'order_id_nunique_x')
(0.0022389062, 'total_cost_sum_x')
(0.0035035694, 'total_cost_min_x')
(0.0024439055, 'total_cost_max_x')
(0.0028178755, 'total_cost_mean_x')
(0.0021086526, 'total_cost_std_x')
(0.002367739, 'rate_mean_x')
(0.0027542536, 'promo_total_sum_x')
(0.0029170369, 'promo_total_mean_x')
(0.0026881993, 'total_weight_sum_x')
(0.0026176842, 'total_weight_mean_x')
(0.007938646, 'month_after_first_x')
(0.002411315, 'order_state_complete_x')
(0.0022735822, 'order_state_canceled_x')
(0.0, 'order_state_resumed_x')
(0.0, 'order_state_cart_x')
(0.0015013054, 'shipment_state_shipped_x')
(0.0, 'shipment_state_canceled_x')
(0.0, 'shipment_state_collecting_x')
(0.0, 'shipment_state_ready_x')
(0.0, 'shipment_state_shipping_x')
(0.0, 'shipment_state_ready_to_ship_x')
(0.0, 'shipment_state_pending_x')
(0.0026869536, 'discount_sum_x')
(0.0030069929, 'quantity_sum_x')
(0.0025099735, 'item_id_nunique_sum_x')
(0.00894188, 'price_sum_x')
(0.002326638, 'price_std_mean_x')
(0.002249

In [9]:
#clf = SGDClassifier()
clf = xgb.XGBClassifier(n_estimators=600, max_depth=6, learning_rate=0.02, random_state=42)

for idx in range(1):
    train_fold = train_folds[idx]
    test_fold = test_folds[idx]
    
    train_features = pd.merge(train_fold, client_features_456, on='phone_id')
    train_features[['rate_mean_new', 'rate_mean_new_x', 'rate_mean_new_y']] = train_features[['rate_mean_new', 'rate_mean_new_x', 'rate_mean_new_y']].fillna(3)
    train_features = train_features.fillna(0)
    print(len(train_fold), len(train_features), train_features.shape)
       
    #print(train_features)
    train_target = deepcopy(train_features['target'])
    del train_features['target']
    del train_features['phone_id']
    del train_features['month_x']
    del train_features['month_y']
    train_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y',
 'shipment_state_canceled_x',
 'shipment_state_canceled_y',
 'shipment_state_canceled',
 'shipment_state_pending_x',
 'shipment_state_pending_y',
 'shipment_state_pending',
 'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship_y',
 'shipment_state_ready_to_ship'
], axis=1, inplace=True)

    
    test_features = pd.merge(test_fold, client_features_567, on='phone_id')
    test_features[['rate_mean_new', 'rate_mean_new_x', 'rate_mean_new_y']] = test_features[
        ['rate_mean_new', 'rate_mean_new_x', 'rate_mean_new_y']].fillna(3)

    test_features = test_features.fillna(0)
    #test_features = pd.merge(test_fold, client_features_7, on='phone_id').fillna(0)
    print(len(test_fold), len(test_features), test_features.shape)
    
    test_target = deepcopy(test_features['target'])
    del test_features['target']
    del test_features['phone_id']
    del test_features['month_x']
    del test_features['month_y']
    test_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y',
 'shipment_state_canceled_x',
 'shipment_state_canceled_y',
 'shipment_state_canceled',
 'shipment_state_pending_x',
 'shipment_state_pending_y',
 'shipment_state_pending',
  'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship_y',
 'shipment_state_ready_to_ship'
], axis=1, inplace=True)
    
    #print(train_features.columns, train_target)
    #print(test_features.columns, test_target)

    clf.fit(train_features.values, train_target)
    print(type(clf))
    new_preds = clf.predict(test_features.values)
    score = f1_score(test_target, new_preds)
    print(score)

84637 75205 (75205, 171)
95052 83455 (83455, 171)
<class 'xgboost.sklearn.XGBClassifier'>
0.7149441744219635


In [19]:
#clf = SGDClassifier()
#clf = xgb.XGBClassifier(n_estimators=500, max_depth=6, learning_rate=0.01, random_state=42)
clf = xgb.XGBClassifier(n_estimators=1000, max_depth=6, learning_rate=0.01, random_state=42)

for idx in range(1):
    train_fold = train_folds[idx]
    test_fold = test_folds[idx]
    
    train_features = pd.merge(train_fold, client_features_456, on='phone_id')
    train_features[['rate_mean_new', 'rate_mean_new_x', 'rate_mean_new_y']] = train_features[['rate_mean_new', 'rate_mean_new_x', 'rate_mean_new_y']].fillna(3)
    train_features = train_features.fillna(0)
    print(len(train_fold), len(train_features), train_features.shape)
       
    #print(train_features)
    train_target = deepcopy(train_features['target'])
    del train_features['target']
    del train_features['phone_id']
    del train_features['month_x']
    del train_features['month_y']
    train_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y',
 'shipment_state_canceled_x',
 'shipment_state_canceled_y',
 'shipment_state_canceled',
 'shipment_state_pending_x',
 'shipment_state_pending_y',
 'shipment_state_pending',
 'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship_y',
 'shipment_state_ready_to_ship'
], axis=1, inplace=True)


    test_features = pd.merge(test_fold, client_features_567, on='phone_id')
    test_features[['rate_mean_new', 'rate_mean_new_x', 'rate_mean_new_y']] = test_features[
        ['rate_mean_new', 'rate_mean_new_x', 'rate_mean_new_y']].fillna(3)

    test_features = test_features.fillna(0)
    #test_features = pd.merge(test_fold, client_features_7, on='phone_id').fillna(0)
    print(len(test_fold), len(test_features), test_features.shape)
    
    test_target = deepcopy(test_features['target'])
    del test_features['target']
    del test_features['phone_id']
    del test_features['month_x']
    del test_features['month_y']
    test_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y',
 'shipment_state_canceled_x',
 'shipment_state_canceled_y',
 'shipment_state_canceled',
 'shipment_state_pending_x',
 'shipment_state_pending_y',
 'shipment_state_pending',
  'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship_y',
 'shipment_state_ready_to_ship'
], axis=1, inplace=True)
    
    #print(train_features.columns, train_target)
    #print(test_features.columns, test_target)

    clf.fit(train_features.values, train_target)
    print(type(clf))
    new_preds = clf.predict(test_features.values)
    score = f1_score(test_target, new_preds)
    print(score)

84637 75205 (75205, 171)
95052 83455 (83455, 171)
<class 'xgboost.sklearn.XGBClassifier'>
0.7152099900752522


In [10]:
for x in zip(clf.feature_importances_, test_features.columns):
    print(x)

(0.0010993681, 'order_id_nunique_x')
(0.0014881849, 'total_cost_sum_x')
(0.0029664766, 'total_cost_min_x')
(0.0013757605, 'total_cost_max_x')
(0.0014705491, 'total_cost_mean_x')
(0.0013260469, 'total_cost_std_x')
(0.001646192, 'rate_mean_x')
(0.0027124924, 'promo_total_sum_x')
(0.0018671148, 'promo_total_mean_x')
(0.001957265, 'total_weight_sum_x')
(0.0067779636, 'total_weight_mean_x')
(0.0053067063, 'month_after_first_x')
(0.0013315694, 'order_state_complete_x')
(0.0013296738, 'order_state_canceled_x')
(0.0, 'order_state_resumed_x')
(0.0, 'order_state_cart_x')
(0.0017956155, 'shipment_state_shipped_x')
(0.0, 'shipment_state_collecting_x')
(0.0, 'shipment_state_ready_x')
(0.0, 'shipment_state_shipping_x')
(0.0016759809, 'discount_sum_x')
(0.0014874305, 'quantity_sum_x')
(0.0019309709, 'item_id_nunique_sum_x')
(0.0059291488, 'price_sum_x')
(0.0015419886, 'price_std_mean_x')
(0.0014509076, 'cancelled_pr_x')
(0.0020021258, 'replaced_pr_x')
(0.0035345925, 'day_after_last_x')
(0.0020583242,

In [9]:
#clf = SGDClassifier()
clf = xgb.XGBClassifier(n_estimators=100, max_depth=14, learning_rate=0.025,
                        random_state=42, base_score=0.6)

for idx in range(2):
    train_fold = train_folds[idx]
    test_fold = test_folds[idx]
    
    train_features = pd.merge(train_fold, client_features_56, on='phone_id').fillna(0)
    #train_features = pd.merge(train_fold, client_features_6, on='phone_id').fillna(0)
    print(len(train_fold), len(train_features), train_features.shape)
       
    #print(train_features)
    train_target = deepcopy(train_features['target'])
    del train_features['target']
    del train_features['phone_id']
    del train_features['month_x']
    del train_features['month_y']
    train_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y',
 'shipment_state_canceled_x',
 'shipment_state_canceled_y',
#  'shipment_state_canceled',
 'shipment_state_pending_x',
 'shipment_state_pending_y',
#  'shipment_state_pending',
 'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship_x',
#  'shipment_state_ready_to_ship'
], axis=1, inplace=True)

    
    test_features = pd.merge(test_fold, client_features_67, on='phone_id').fillna(0)
    #test_features = pd.merge(test_fold, client_features_7, on='phone_id').fillna(0)
    print(len(test_fold), len(test_features), test_features.shape)
    
    test_target = deepcopy(test_features['target'])
    del test_features['target']
    del test_features['phone_id']
    del test_features['month_x']
    del test_features['month_y']
    test_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y',
 'shipment_state_canceled_x',
 'shipment_state_canceled_y',
#  'shipment_state_canceled',
 'shipment_state_pending_x',
 'shipment_state_pending_y',
#  'shipment_state_pending',
  'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship_x',
#  'shipment_state_ready_to_ship'
], axis=1, inplace=True)
    
    #print(train_features.columns, train_target)
    #print(test_features.columns, test_target)

    clf.fit(train_features, train_target)
    print(type(clf))
    new_preds = clf.predict(test_features)
    score = f1_score(test_target, new_preds)
    print(score)

84907 69543 (69543, 109)
94722 70824 (70824, 109)
<class 'xgboost.sklearn.XGBClassifier'>
0.7296406071570628
84600 69412 (69412, 109)
94253 70683 (70683, 109)


KeyboardInterrupt: 

In [15]:
#clf = SGDClassifier()
clf = xgb.XGBClassifier(n_estimators=1300, max_depth=6, learning_rate=0.02, random_state=42)

for idx in range(3):
    train_fold = train_folds[idx]
    test_fold = test_folds[idx]
    
    train_features = pd.merge(train_fold, client_features_456, on='phone_id').fillna(0)
    #train_features = pd.merge(train_fold, client_features_6, on='phone_id').fillna(0)
    print(len(train_fold), len(train_features), train_features.shape)
       
    #print(train_features)
    train_target = deepcopy(train_features['target'])
    del train_features['target']
    del train_features['phone_id']
    del train_features['month_x']
    del train_features['month_y']
    train_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y',
 'shipment_state_canceled_x',
 'shipment_state_canceled_y',
 'shipment_state_canceled',
 'shipment_state_pending_x',
 'shipment_state_pending_y',
 'shipment_state_pending',
 'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship'
], axis=1, inplace=True)

    
    test_features = pd.merge(test_fold, client_features_567, on='phone_id').fillna(0)
    #test_features = pd.merge(test_fold, client_features_7, on='phone_id').fillna(0)
    print(len(test_fold), len(test_features), test_features.shape)
    
    test_target = deepcopy(test_features['target'])
    del test_features['target']
    del test_features['phone_id']
    del test_features['month_x']
    del test_features['month_y']
    test_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y',
 'shipment_state_canceled_x',
 'shipment_state_canceled_y',
 'shipment_state_canceled',
 'shipment_state_pending_x',
 'shipment_state_pending_y',
 'shipment_state_pending',
  'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship'
], axis=1, inplace=True)
    
    #print(train_features.columns, train_target)
    #print(test_features.columns, test_target)

    clf.fit(train_features, train_target)
    print(type(clf))
    new_preds = clf.predict(test_features)
    score = f1_score(test_target, new_preds)
    print(score)

84880 75515 (75515, 138)
94738 83301 (83301, 138)
<class 'xgboost.sklearn.XGBClassifier'>
0.7152541103329464
84734 75160 (75160, 138)
94577 82819 (82819, 138)
<class 'xgboost.sklearn.XGBClassifier'>
0.713989584913292
84671 75222 (75222, 138)
95134 83538 (83538, 138)


KeyboardInterrupt: 

In [ ]:
#clf = SGDClassifier()
clf = xgb.XGBClassifier(n_estimators=1600, max_depth=6, learning_rate=0.01, random_state=42)

for idx in range(3):
    train_fold = train_folds[idx]
    test_fold = test_folds[idx]
    
    train_features = pd.merge(train_fold, client_features_456, on='phone_id').fillna(0)
    #train_features = pd.merge(train_fold, client_features_6, on='phone_id').fillna(0)
    print(len(train_fold), len(train_features), train_features.shape)
       
    #print(train_features)
    train_target = deepcopy(train_features['target'])
    del train_features['target']
    del train_features['phone_id']
    del train_features['month_x']
    del train_features['month_y']
    train_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y',
 'shipment_state_canceled_x',
 'shipment_state_canceled_y',
 'shipment_state_canceled',
 'shipment_state_pending_x',
 'shipment_state_pending_y',
 'shipment_state_pending',
 'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship'
], axis=1, inplace=True)

    
    test_features = pd.merge(test_fold, client_features_567, on='phone_id').fillna(0)
    #test_features = pd.merge(test_fold, client_features_7, on='phone_id').fillna(0)
    print(len(test_fold), len(test_features), test_features.shape)
    
    test_target = deepcopy(test_features['target'])
    del test_features['target']
    del test_features['phone_id']
    del test_features['month_x']
    del test_features['month_y']
    test_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y',
 'shipment_state_canceled_x',
 'shipment_state_canceled_y',
 'shipment_state_canceled',
 'shipment_state_pending_x',
 'shipment_state_pending_y',
 'shipment_state_pending',
  'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship'
], axis=1, inplace=True)
    
    #print(train_features.columns, train_target)
    #print(test_features.columns, test_target)

    clf.fit(train_features, train_target)
    print(type(clf))
    new_preds = clf.predict(test_features)
    score = f1_score(test_target, new_preds)
    print(score)

In [6]:
from sklearn.ensemble import RandomForestClassifier

In [11]:
#clf = SGDClassifier()
clf = RandomForestClassifier(n_estimators=600,
                              #criterion='entropy',
                              max_depth=None, 
                              random_state=42,
                             verbose=2,
                             n_jobs=7
                            )
for idx in range(3):
    train_fold = train_folds[idx]
    test_fold = test_folds[idx]
    
    train_features = pd.merge(train_fold, client_features_456, on='phone_id').fillna(0)
    #train_features = pd.merge(train_fold, client_features_6, on='phone_id').fillna(0)
    print(len(train_fold), len(train_features), train_features.shape)
       
    #print(train_features)
    train_target = deepcopy(train_features['target'])
    del train_features['target']
    del train_features['phone_id']
    del train_features['month_x']
    del train_features['month_y']
    train_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y',
 'shipment_state_canceled_x',
 'shipment_state_canceled_y',
 'shipment_state_canceled',
 'shipment_state_pending_x',
 'shipment_state_pending_y',
 'shipment_state_pending',
 'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship'
], axis=1, inplace=True)

    
    test_features = pd.merge(test_fold, client_features_567, on='phone_id').fillna(0)
    print(len(test_fold), len(test_features), test_features.shape)
    
    test_target = deepcopy(test_features['target'])
    del test_features['target']
    del test_features['phone_id']
    del test_features['month_x']
    del test_features['month_y']
    test_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y',
 'shipment_state_canceled_x',
 'shipment_state_canceled_y',
 'shipment_state_canceled',
 'shipment_state_pending_x',
 'shipment_state_pending_y',
 'shipment_state_pending',
  'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship'
], axis=1, inplace=True)
    
    #print(train_features.columns, train_target)
    #print(test_features.columns, test_target)

    clf.fit(train_features, train_target)
    print(type(clf))
    new_preds = clf.predict(test_features)
    score = f1_score(test_target, new_preds)
    print(score)

84361 75045 (75045, 162)
94432 82910 (82910, 162)


[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.


building tree 1 of 600
building tree 2 of 600
building tree 3 of 600
building tree 4 of 600
building tree 5 of 600
building tree 6 of 600
building tree 7 of 600
building tree 8 of 600
building tree 9 of 600
building tree 10 of 600
building tree 11 of 600
building tree 12 of 600
building tree 13 of 600
building tree 14 of 600
building tree 15 of 600
building tree 16 of 600
building tree 17 of 600
building tree 18 of 600
building tree 19 of 600
building tree 20 of 600
building tree 21 of 600
building tree 22 of 600
building tree 23 of 600
building tree 24 of 600
building tree 25 of 600
building tree 26 of 600
building tree 27 of 600
building tree 28 of 600
building tree 29 of 600
building tree 30 of 600
building tree 31 of 600
building tree 32 of 600
building tree 33 of 600


[Parallel(n_jobs=7)]: Done  27 tasks      | elapsed:    1.7s


building tree 34 of 600
building tree 35 of 600
building tree 36 of 600
building tree 37 of 600
building tree 38 of 600
building tree 39 of 600
building tree 40 of 600
building tree 41 of 600
building tree 42 of 600
building tree 43 of 600
building tree 44 of 600
building tree 45 of 600
building tree 46 of 600
building tree 47 of 600
building tree 48 of 600
building tree 49 of 600
building tree 50 of 600
building tree 51 of 600
building tree 52 of 600
building tree 53 of 600
building tree 54 of 600
building tree 55 of 600
building tree 56 of 600
building tree 57 of 600
building tree 58 of 600
building tree 59 of 600
building tree 60 of 600
building tree 61 of 600
building tree 62 of 600
building tree 63 of 600
building tree 64 of 600
building tree 65 of 600
building tree 66 of 600
building tree 67 of 600
building tree 68 of 600building tree 69 of 600

building tree 70 of 600
building tree 71 of 600
building tree 72 of 600
building tree 73 of 600
building tree 74 of 600
building tree 75

...

[Parallel(n_jobs=7)]: Done 600 out of 600 | elapsed:   42.2s finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.


<class 'sklearn.ensemble._forest.RandomForestClassifier'>


[Parallel(n_jobs=7)]: Done  27 tasks      | elapsed:    0.2s
[Parallel(n_jobs=7)]: Done 148 tasks      | elapsed:    1.0s
[Parallel(n_jobs=7)]: Done 351 tasks      | elapsed:    2.3s


0.7172272384787363


[Parallel(n_jobs=7)]: Done 600 out of 600 | elapsed:    3.8s finished


In [13]:
#clf = SGDClassifier()
clf = RandomForestClassifier(n_estimators=800,
                              #criterion='entropy',
                              max_depth=None, 
                              random_state=42,
                             verbose=2
                            )
for idx in range(3):
    train_fold = train_folds[idx]
    test_fold = test_folds[idx]
    
    train_features = pd.merge(train_fold, client_features_456, on='phone_id').fillna(0)
    #train_features = pd.merge(train_fold, client_features_6, on='phone_id').fillna(0)
    print(len(train_fold), len(train_features), train_features.shape)
       
    #print(train_features)
    train_target = deepcopy(train_features['target'])
    del train_features['target']
    del train_features['phone_id']
    del train_features['month_x']
    del train_features['month_y']
    train_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y',
 'shipment_state_canceled_x',
 'shipment_state_canceled_y',
 'shipment_state_canceled',
 'shipment_state_pending_x',
 'shipment_state_pending_y',
 'shipment_state_pending',
 'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship'
], axis=1, inplace=True)

    
    test_features = pd.merge(test_fold, client_features_567, on='phone_id').fillna(0)
    print(len(test_fold), len(test_features), test_features.shape)
    
    test_target = deepcopy(test_features['target'])
    del test_features['target']
    del test_features['phone_id']
    del test_features['month_x']
    del test_features['month_y']
    test_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y',
 'shipment_state_canceled_x',
 'shipment_state_canceled_y',
 'shipment_state_canceled',
 'shipment_state_pending_x',
 'shipment_state_pending_y',
 'shipment_state_pending',
  'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship'
], axis=1, inplace=True)
    
    #print(train_features.columns, train_target)
    #print(test_features.columns, test_target)

    clf.fit(train_features, train_target)
    print(type(clf))
    new_preds = clf.predict(test_features)
    score = f1_score(test_target, new_preds)
    print(score)

84885 75371 (75371, 138)
94730 83105 (83105, 138)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 800


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


building tree 2 of 800
building tree 3 of 800
building tree 4 of 800
building tree 5 of 800
building tree 6 of 800
building tree 7 of 800
building tree 8 of 800
building tree 9 of 800
building tree 10 of 800
building tree 11 of 800
building tree 12 of 800
building tree 13 of 800
building tree 14 of 800
building tree 15 of 800
building tree 16 of 800
building tree 17 of 800
building tree 18 of 800
building tree 19 of 800
building tree 20 of 800
building tree 21 of 800
building tree 22 of 800
building tree 23 of 800
building tree 24 of 800
building tree 25 of 800
building tree 26 of 800
building tree 27 of 800
building tree 28 of 800
building tree 29 of 800
building tree 30 of 800
building tree 31 of 800
building tree 32 of 800
building tree 33 of 800
building tree 34 of 800
building tree 35 of 800
building tree 36 of 800
building tree 37 of 800
building tree 38 of 800
building tree 39 of 800
building tree 40 of 800
building tree 41 of 800
building tree 42 of 800
building tree 43 of 800


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 800


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


building tree 2 of 800
building tree 3 of 800
building tree 4 of 800
building tree 5 of 800
building tree 6 of 800
building tree 7 of 800
building tree 8 of 800
building tree 9 of 800
building tree 10 of 800
building tree 11 of 800
building tree 12 of 800
building tree 13 of 800
building tree 14 of 800
building tree 15 of 800
building tree 16 of 800
building tree 17 of 800
building tree 18 of 800
building tree 19 of 800
building tree 20 of 800
building tree 21 of 800
building tree 22 of 800
building tree 23 of 800
building tree 24 of 800
building tree 25 of 800
building tree 26 of 800
building tree 27 of 800
building tree 28 of 800
building tree 29 of 800
building tree 30 of 800
building tree 31 of 800
building tree 32 of 800
building tree 33 of 800
building tree 34 of 800
building tree 35 of 800
building tree 36 of 800
building tree 37 of 800
building tree 38 of 800
building tree 39 of 800
building tree 40 of 800
building tree 41 of 800
building tree 42 of 800
building tree 43 of 800


KeyboardInterrupt: 

In [22]:
from sklearn.neighbors import KNeighborsClassifier

In [26]:
#clf = SGDClassifier()
clf = KNeighborsClassifier(n_neighbors=200)

for idx in range(3):
    train_fold = train_folds[idx]
    test_fold = test_folds[idx]
    
    train_features = pd.merge(train_fold, client_features_456, on='phone_id').fillna(0)
    #train_features = pd.merge(train_fold, client_features_6, on='phone_id').fillna(0)
    print(len(train_fold), len(train_features), train_features.shape)
       
    #print(train_features)
    train_target = deepcopy(train_features['target'])
    del train_features['target']
    del train_features['phone_id']
    del train_features['month_x']
    del train_features['month_y']
    train_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y',
 'shipment_state_canceled_x',
 'shipment_state_canceled_y',
 'shipment_state_canceled',
 'shipment_state_pending_x',
 'shipment_state_pending_y',
 'shipment_state_pending',
 'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship'
], axis=1, inplace=True)

    
    test_features = pd.merge(test_fold, client_features_567, on='phone_id').fillna(0)
    print(len(test_fold), len(test_features), test_features.shape)
    
    test_target = deepcopy(test_features['target'])
    del test_features['target']
    del test_features['phone_id']
    del test_features['month_x']
    del test_features['month_y']
    test_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y',
 'shipment_state_canceled_x',
 'shipment_state_canceled_y',
 'shipment_state_canceled',
 'shipment_state_pending_x',
 'shipment_state_pending_y',
 'shipment_state_pending',
  'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship'
], axis=1, inplace=True)
    
    #print(train_features.columns, train_target)
    #print(test_features.columns, test_target)

    clf.fit(train_features, train_target)
    print(type(clf))
    new_preds = clf.predict(test_features)
    score = f1_score(test_target, new_preds)
    print(score)

84780 75235 (75235, 162)
94986 83322 (83322, 162)
<class 'sklearn.neighbors._classification.KNeighborsClassifier'>
0.6946091307066917
84809 75564 (75564, 162)
94893 83336 (83336, 162)
<class 'sklearn.neighbors._classification.KNeighborsClassifier'>


KeyboardInterrupt: 

In [14]:
for x in zip(clf.feature_importances_, test_features.columns):
    print(x)

(0.0, 'order_id_nunique_x')
(0.0, 'total_cost_sum_x')
(0.0, 'total_cost_min_x')
(0.0, 'total_cost_max_x')
(0.0, 'total_cost_mean_x')
(0.0, 'total_cost_std_x')
(0.0, 'rate_mean_x')
(0.0, 'promo_total_sum_x')
(0.0, 'promo_total_mean_x')
(0.0, 'total_weight_sum_x')
(0.0, 'total_weight_mean_x')
(0.0, 'month_after_first_x')
(0.0, 'order_state_complete_x')
(0.0, 'order_state_canceled_x')
(0.0, 'order_state_resumed_x')
(0.0, 'order_state_cart_x')
(0.0, 'shipment_state_shipped_x')
(0.0, 'shipment_state_collecting_x')
(0.0, 'shipment_state_ready_x')
(0.0, 'shipment_state_shipping_x')
(0.0, 'discount_sum_x')
(0.0, 'quantity_sum_x')
(0.0, 'item_id_nunique_sum_x')
(0.0, 'price_sum_x')
(0.0, 'price_std_mean_x')
(0.0, 'cancelled_pr_x')
(0.0, 'replaced_pr_x')
(0.0, 'day_after_last_x')
(0.0, 'android_x')
(0.0, 'ios_x')
(0.0, 'linux_x')
(0.0, 'mac_x')
(0.0, 'other_x')
(0.0, 'windows_x')
(0.0, 'Other_city_x')
(0.0, 'Москва_x')
(0.0, 'Санкт-Петербург_x')
(0.0, 'order_id_nunique_y')
(0.0, 'total_cost_sum_

In [18]:
from sklearn.neural_network import MLPClassifier

In [23]:
from sklearn.preprocessing import maxabs_scale

In [7]:
result_clf = RandomForestClassifier(n_estimators=800,
                              #criterion='entropy',
                              max_depth=None, 
                              random_state=42,
                             verbose=2, n_jobs=7
                            )

test_features = pd.merge(test_data, client_features_567, on='phone_id').fillna(0)

print(len(test_data), len(test_features), test_features.shape)
    
test_target = deepcopy(test_features['target'])
del test_features['target']
del test_features['fold_num']
del test_features['phone_id']
del test_features['month_x']
del test_features['month_y']
del test_features['order_completed_at']
test_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y',
  'shipment_state_canceled_x',
 'shipment_state_canceled_y',
 'shipment_state_canceled',
 'shipment_state_pending_x',
 'shipment_state_pending_y',
 'shipment_state_pending',
  'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship_y',
 'shipment_state_ready_to_ship'
], axis=1, inplace=True)
    
    #print(train_features.columns, train_target)
    #print(test_features.columns, test_target)

result_clf.fit(test_features, test_target)

284449 249658 (249658, 164)


[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.


building tree 1 of 800building tree 2 of 800

building tree 3 of 800
building tree 4 of 800
building tree 5 of 800
building tree 6 of 800
building tree 7 of 800
building tree 8 of 800
building tree 9 of 800
building tree 10 of 800
building tree 11 of 800
building tree 12 of 800
building tree 13 of 800
building tree 14 of 800
building tree 15 of 800
building tree 16 of 800
building tree 17 of 800
building tree 18 of 800
building tree 19 of 800
building tree 20 of 800
building tree 21 of 800
building tree 22 of 800
building tree 23 of 800
building tree 24 of 800
building tree 25 of 800
building tree 26 of 800
building tree 27 of 800
building tree 28 of 800
building tree 29 of 800
building tree 30 of 800
building tree 31 of 800
building tree 32 of 800
building tree 33 of 800


[Parallel(n_jobs=7)]: Done  27 tasks      | elapsed:    7.9s


building tree 34 of 800
building tree 35 of 800
building tree 36 of 800
building tree 37 of 800
building tree 38 of 800
building tree 39 of 800
building tree 40 of 800
building tree 41 of 800
building tree 42 of 800
building tree 43 of 800
building tree 44 of 800
building tree 45 of 800
building tree 46 of 800
building tree 47 of 800
building tree 48 of 800
building tree 49 of 800
building tree 50 of 800
building tree 51 of 800
building tree 52 of 800
building tree 53 of 800
building tree 54 of 800
building tree 55 of 800
building tree 56 of 800
building tree 57 of 800
building tree 58 of 800
building tree 59 of 800
building tree 60 of 800
building tree 61 of 800
building tree 62 of 800
building tree 63 of 800
building tree 64 of 800
building tree 65 of 800
building tree 66 of 800
building tree 67 of 800
building tree 68 of 800
building tree 69 of 800
building tree 70 of 800
building tree 71 of 800
building tree 72 of 800
building tree 73 of 800
building tree 74 of 800
building tree 75

...

building tree 358 of 800
building tree 359 of 800
building tree 360 of 800
building tree 361 of 800
building tree 362 of 800
building tree 363 of 800
building tree 364 of 800
building tree 365 of 800
building tree 366 of 800
building tree 367 of 800
building tree 368 of 800
building tree 369 of 800
building tree 370 of 800
building tree 371 of 800
building tree 372 of 800
building tree 373 of 800
building tree 374 of 800
building tree 375 of 800
building tree 376 of 800
building tree 377 of 800
building tree 378 of 800
building tree 379 of 800
building tree 380 of 800
building tree 381 of 800
building tree 382 of 800
building tree 383 of 800
building tree 384 of 800
building tree 385 of 800
building tree 386 of 800
building tree 387 of 800
building tree 388 of 800
building tree 389 of 800
building tree 390 of 800
building tree 391 of 800
building tree 392 of 800
building tree 393 of 800
building tree 394 of 800
building tree 395 of 800
building tree 396 of 800
building tree 397 of 800


[Parallel(n_jobs=7)]: Done 634 tasks      | elapsed:  2.9min


building tree 641 of 800
building tree 642 of 800
building tree 643 of 800
building tree 644 of 800
building tree 645 of 800
building tree 646 of 800
building tree 647 of 800
building tree 648 of 800
building tree 649 of 800
building tree 650 of 800
building tree 651 of 800
building tree 652 of 800
building tree 653 of 800
building tree 654 of 800
building tree 655 of 800
building tree 656 of 800
building tree 657 of 800
building tree 658 of 800
building tree 659 of 800
building tree 660 of 800
building tree 661 of 800
building tree 662 of 800
building tree 663 of 800
building tree 664 of 800
building tree 665 of 800
building tree 666 of 800
building tree 667 of 800
building tree 668 of 800
building tree 669 of 800
building tree 670 of 800
building tree 671 of 800
building tree 672 of 800
building tree 673 of 800
building tree 674 of 800
building tree 675 of 800
building tree 676 of 800
building tree 677 of 800
building tree 678 of 800
building tree 679 of 800
building tree 680 of 800


[Parallel(n_jobs=7)]: Done 800 out of 800 | elapsed:  3.8min finished


RandomForestClassifier(n_estimators=800, n_jobs=7, random_state=42, verbose=2)

In [8]:
subm_features = pd.merge(df_test, client_features_678, how='left', 
                         left_on='Id', right_on='phone_id').fillna(0)
del subm_features['Predicted']
del subm_features['Id']
del subm_features['phone_id']
del subm_features['month_x']
del subm_features['month_y']
del subm_features['month']
#del subm_features['order_completed_at']
subm_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y',
 'shipment_state_canceled_x',
 'shipment_state_canceled_y',
 'shipment_state_canceled',
 'shipment_state_pending_x',
 'shipment_state_pending_y',
 'shipment_state_pending',
  'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship_y',
 'shipment_state_ready_to_ship'
], axis=1, inplace=True)

result_preds = result_clf.predict(subm_features)
df_test['Predicted'] = result_preds
df_test['Predicted'] = df_test['Predicted'].astype(int)
df_test.to_csv("./HackIt_3_month_submit_new_feat_rf.csv", index=False)

[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  27 tasks      | elapsed:    0.7s
[Parallel(n_jobs=7)]: Done 148 tasks      | elapsed:    3.7s
[Parallel(n_jobs=7)]: Done 351 tasks      | elapsed:    8.4s
[Parallel(n_jobs=7)]: Done 634 tasks      | elapsed:   15.0s
[Parallel(n_jobs=7)]: Done 800 out of 800 | elapsed:   18.8s finished


In [9]:
df_test['Predicted'] = result_preds
df_test['Predicted'] = df_test['Predicted'].astype(int)
df_test.to_csv("./HackIt_3_month_submit_new_feat_rf.csv", index=False)

In [14]:
result_clf = xgb.XGBClassifier(n_estimators=100, max_depth=14, learning_rate=0.025,
                        random_state=42, base_score=0.6)

test_features = pd.merge(test_data, client_features_67, on='phone_id').fillna(0)

print(len(test_data), len(test_features), test_features.shape)
    
test_target = deepcopy(test_features['target'])
del test_features['target']
del test_features['fold_num']
del test_features['phone_id']
del test_features['month_x']
del test_features['month_y']
del test_features['month']
del test_features['order_completed_at']
test_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y',
  'shipment_state_canceled_x',
 'shipment_state_canceled_y',
#  'shipment_state_canceled',
 'shipment_state_pending_x',
 'shipment_state_pending_y',
#  'shipment_state_pending',
  'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship_x',
#  'shipment_state_ready_to_ship'
], axis=1, inplace=True)
    
    #print(train_features.columns, train_target)
    #print(test_features.columns, test_target)

result_clf.fit(test_features, test_target)

284449 212827 (212827, 111)


XGBClassifier(base_score=0.6, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.025, max_delta_step=0, max_depth=14,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [15]:
subm_features = pd.merge(df_test, client_features_78, how='left', 
                         left_on='Id', right_on='phone_id').fillna(0)
del subm_features['Predicted']
del subm_features['Id']
del subm_features['phone_id']
del subm_features['month_x']
del subm_features['month_y']
#del subm_features['month']
#del subm_features['order_completed_at']
subm_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y',
 'shipment_state_canceled_x',
 'shipment_state_canceled_y',
#  'shipment_state_canceled',
 'shipment_state_pending_x',
 'shipment_state_pending_y',
#  'shipment_state_pending',
  'shipment_state_ready_to_ship_x',
 'shipment_state_ready_to_ship_x',
#  'shipment_state_ready_to_ship'
], axis=1, inplace=True)

result_preds = result_clf.predict(subm_features)
df_test['Predicted'] = result_preds
df_test['Predicted'] = df_test['Predicted'].astype(int)
df_test.to_csv("./HackIt_2_month_submit_new_feat_xgb.csv", index=False)
sum(result_preds)

117365.0

In [ ]:
result_clf = xgb.XGBClassifier(n_estimators=1000, max_depth=6, learning_rate=0.01, random_state=42)

    
test_features = pd.merge(test_data, client_features_567, on='phone_id').fillna(0)
test_features[['rate_mean_new', 'rate_mean_new_x', 'rate_mean_new_y']] = test_features[
    ['rate_mean_new', 'rate_mean_new_x', 'rate_mean_new_y']].fillna(3)
del test_features['month_y']

test_features = test_features.fillna(0)
#test_features = pd.merge(test_fold, client_features_7, on='phone_id').fillna(0)
print(len(test_fold), len(test_features), test_features.shape)

test_target = deepcopy(test_features['target'])
del test_features['target']
del test_features['phone_id']
del test_features['month_x']
del test_features['month_y']
del test_features['fold_num']
del test_features['order_completed_at']
test_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y',
 'shipment_state_canceled_x',
 'shipment_state_canceled_y',
 'shipment_state_canceled',
 'shipment_state_pending_x',
 'shipment_state_pending_y',
 'shipment_state_pending',
  'shipment_state_ready_to_ship_x',

 'shipment_state_ready_to_ship_y',
 'shipment_state_ready_to_ship'
], axis=1, inplace=True)

result_clf.fit(test_features, test_target)

In [25]:
subm_features = pd.merge(df_test, client_features_678, how='left', 
                         left_on='Id', right_on='phone_id').fillna(0)
del subm_features['Predicted']
del subm_features['Id']
del subm_features['phone_id']
del subm_features['month_x']
del subm_features['month_y']
del subm_features['month']
#del subm_features['order_completed_at']
subm_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y',
 'shipment_state_canceled_x',
 'shipment_state_canceled_y',
 'shipment_state_canceled',
 'shipment_state_pending_x',
 'shipment_state_pending_y',
 'shipment_state_pending',
  'shipment_state_ready_to_ship_x',

 'shipment_state_ready_to_ship_y',
 'shipment_state_ready_to_ship'], axis=1, inplace=True)

result_preds = result_clf.predict(subm_features)
df_test['Predicted'] = result_preds
df_test['Predicted'] = df_test['Predicted'].astype(int)
df_test.to_csv("./HackIt_3_month_submit_super_new_feat.csv", index=False)
print(sum(result_preds))

114322.0


In [20]:
result_clf = xgb.XGBClassifier(n_estimators=500, max_depth=6, learning_rate=0.05)

test_features = pd.merge(test_data, client_features_567, on='phone_id').fillna(0)
test_features_prev = pd.merge(test_data, client_features_456, on='phone_id').fillna(0)
test_features = pd.concat([test_features, test_features_prev])
print(len(test_data), len(test_features), test_features.shape)
    
test_target = deepcopy(test_features['target'])
del test_features['target']
del test_features['fold_num']
del test_features['phone_id']
del test_features['month_x']
del test_features['month_y']
del test_features['order_completed_at']
test_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y'], axis=1, inplace=True)
    
    #print(train_features.columns, train_target)
    #print(test_features.columns, test_target)

result_clf.fit(test_features, test_target)

284449 475736 (475736, 164)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=500, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [21]:
subm_features = pd.merge(df_test, client_features_678, how='left', 
                         left_on='Id', right_on='phone_id').fillna(0)
del subm_features['Predicted']
del subm_features['Id']
del subm_features['phone_id']
del subm_features['month_x']
del subm_features['month_y']
del subm_features['month']
#del subm_features['order_completed_at']
subm_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y'], axis=1, inplace=True)

result_preds = result_clf.predict(subm_features)
df_test['Predicted'] = result_preds
df_test['Predicted'] = df_test['Predicted'].astype(int)
df_test.to_csv("./HackIt_3_month_submit_new_feat_two_period_in_train.csv", index=False)
print(sum(result_preds))

101668.0


In [29]:
result_clf = KNeighborsClassifier(n_neighbors=200)

test_features = pd.merge(test_data, client_features_567, on='phone_id').fillna(0)
print(len(test_data), len(test_features), test_features.shape)
    
test_target = deepcopy(test_features['target'])
del test_features['target']
del test_features['fold_num']
del test_features['phone_id']
del test_features['month_x']
del test_features['month_y']
del test_features['order_completed_at']
test_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x',
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y'], axis=1, inplace=True)
    
    #print(train_features.columns, train_target)
    #print(test_features.columns, test_target)

result_clf.fit(test_features, test_target)

284449 249658 (249658, 164)


KNeighborsClassifier(n_neighbors=200)

In [30]:
subm_features = pd.merge(df_test, client_features_678, how='left', 
                         left_on='Id', right_on='phone_id').fillna(0)
del subm_features['Predicted']
del subm_features['Id']
del subm_features['phone_id']
del subm_features['month_x']
del subm_features['month_y']
del subm_features['month']
#del subm_features['order_completed_at']
subm_features.drop([
 'age_x',
 'age_y',
 'gender_female_x',
 'gender_female_y',
 'gender_male_x', 
 'gender_male_y',
 'genger_nan_x',
 'genger_nan_y'], axis=1, inplace=True)

result_preds = result_clf.predict(subm_features)
df_test['Predicted'] = result_preds
df_test['Predicted'] = df_test['Predicted'].astype(int)
df_test.to_csv("./HackIt_3_month_submit_new_feat_knn.csv", index=False)
print(sum(result_preds))

118394.0


In [ ]:
corr = test_features.corr()
corr.style.background_gradient(cmap='coolwarm')

In [10]:
sum(result_preds)

113048.0

In [12]:
result_clf.feature_importances_

array([4.15511207e-03, 8.06193261e-03, 5.51459370e-03, 6.07318931e-03,
       8.24796969e-03, 4.97508704e-03, 5.76211164e-03, 4.36697258e-03,
       4.57620633e-03, 1.53897591e-02, 1.52748582e-02, 6.48137756e-03,
       4.41288784e-03, 1.17130360e-03, 9.25543035e-05, 0.00000000e+00,
       4.36147424e-03, 7.81467669e-06, 2.14020222e-05, 2.22872940e-05,
       1.44798786e-02, 1.43529639e-02, 1.40012994e-02, 1.53168925e-02,
       1.53566295e-02, 1.12451785e-02, 1.17661927e-02, 1.26579851e-02,
       2.05244723e-03, 1.89389894e-03, 1.43258535e-04, 6.65384380e-04,
       4.60001938e-05, 1.43212794e-03, 1.90778819e-03, 1.63796480e-03,
       9.49999150e-04, 0.00000000e+00, 0.00000000e+00, 8.51758262e-03,
       4.71170923e-03, 7.52502072e-03, 0.00000000e+00, 8.40629249e-03,
       2.58441118e-03, 5.13572890e-03, 7.91027058e-03, 5.32555099e-03,
       5.70837810e-03, 7.59402561e-03, 4.82585549e-03, 6.13080552e-03,
       5.33948620e-03, 5.46957168e-03, 1.76231420e-02, 1.60948580e-02,
      

In [16]:
for x in zip(result_clf.feature_importances_, test_features.columns):
    print(x)

(0.002658316, 'order_id_nunique_x')
(0.0020587794, 'total_cost_sum_x')
(0.0023708122, 'total_cost_min_x')
(0.0021963664, 'total_cost_max_x')
(0.0022289199, 'total_cost_mean_x')
(0.0020998092, 'total_cost_std_x')
(0.0024504827, 'rate_mean_x')
(0.0020596855, 'promo_total_sum_x')
(0.002216425, 'promo_total_mean_x')
(0.0025330016, 'total_weight_sum_x')
(0.0022313225, 'total_weight_mean_x')
(0.016270176, 'month_after_first_x')
(0.006470543, 'order_state_complete_x')
(0.0020771066, 'order_state_canceled_x')
(0.002053081, 'order_state_resumed_x')
(0.0, 'order_state_cart_x')
(0.010442407, 'shipment_state_shipped_x')
(0.0, 'shipment_state_collecting_x')
(0.0, 'shipment_state_ready_x')
(0.0, 'shipment_state_shipping_x')
(0.002197464, 'discount_sum_x')
(0.0025991665, 'quantity_sum_x')
(0.0020927833, 'item_id_nunique_sum_x')
(0.002371797, 'price_sum_x')
(0.0021378314, 'price_std_mean_x')
(0.002065635, 'cancelled_pr_x')
(0.0020817441, 'replaced_pr_x')
(0.0033672594, 'day_after_last_x')
(0.001925681